In [1]:
import pandas as pd
import os
import time
import csv
import re
import codecs
import requests
from bs4 import BeautifulSoup
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException

In [2]:
def text_file_to_dataframe(filename):
    # Check if the text file exists
    if os.path.exists(filename):
        
        # Read the text file and create a list of lines
        with open(filename, "r") as file:
            lines = file.readlines()
            content = [line.strip() for line in lines]

        # Create a DataFrame from the list
        df = pd.DataFrame(content, columns=['links'])
        
        return df
    else:
        print(f"{filename} does not exist.")
        return None

# Loading Links & Creating DataFrame

In [3]:
input_filename = "links.txt"
df = text_file_to_dataframe(input_filename)

In [6]:
df = pd.read_csv('reviews.csv')

In [7]:
df

,Book Name,page_count,year,review_count,rating,rating_count,genre,author,Review 1,Review 2,Review 3,Review 4,Review 5
0,The Godmothers #5\nBreaking News,"272 pages, Paperback","First published January 1, 2012",131,4.17,"1,649 ratings",Romance,Fern Michaels,Not the greatest until about the last third of...,This is the fifth sequel in this series. A lit...,This book reminded me of The Golden Girls tele...,fern michaels never fails to provide a story t...,Another good read!
1,"Jokes, Jokes and More Jokes: Hilarious Adult H...","146 pages, Paperback","Published July 16, 2015",0,3.00,1 rating,NaN,Joke Star Funny Bones Mr.,NaN,NaN,NaN,NaN,NaN
2,Millie's Tropical World,"96 pages, Paperback","First published September 1, 2015",2,4.49,87 ratings,Colouring Books,Millie Marotta,Check out this and other reviews on my young a...,Loved spending time colouring this adult colou...,NaN,NaN,NaN
3,Legally Stoned: The Most Effective Substances ...,"62 pages, Kindle Edition","First published July 17, 2015",1,3.75,4 ratings,NaN,Micheal Kratom,Interesting book!,NaN,NaN,NaN,NaN
4,Secret Lives of the First Ladies,"296 pages, Paperback","First published May 1, 2005",206,3.79,"1,823 ratings",History,Cormac O'Brien,I received this book for free from the publish...,I'm giving this a five star since I'd really o...,There isn't so much plot in this book as there...,"Well, that was fun.\n\nMy f-t-f book club is r...",This was different than the stories you get of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,First Man: Reimagining Matthew Henson,"164 pages, Hardcover","First published April 1, 2012",61,3.70,279 ratings,Graphic Novels,Simon Schwartz,First Man is largely about a black man named M...,If you go to the last couple pages of this boo...,Matthew Henson was one of the first men to rea...,"If I had an endless supply of money, I would b...","I liked a great deal about this book, includin..."
468,Birth Control,"128 pages, Hardcover","First published January 1, 2009",1,1.00,1 rating,NaN,Lauri S. Friedman,I was disapointed. I had enjoyed the last oppo...,NaN,NaN,NaN,NaN
469,Crafting Short Screenplays That Connect,Kindle Edition,"Published January 21, 2005",0,0.00,0 ratings,NaN,Claudia H. Johnson,NaN,NaN,NaN,NaN,NaN
470,God Loves Hair,"99 pages, Paperback","First published May 1, 2010",98,4.02,717 ratings,Short Stories,Vivek Shraya,God Loves Hair by Vivek Shraya is a very impac...,It's hard to believe these are Vivek Shraya's ...,"An interesting collection of stories, as colou...",Tiny glimpses on the writers life. It was writ...,Thanks to Netgalley for the opportunity to rea...


# Scraper Code

In [4]:
# S C R A P I N G   B R O W S E R
driver = webdriver.Chrome()

reviews_file_exists = os.path.isfile("reviews.csv")
print("Does Review CSV File Exists?")
print(reviews_file_exists)
print("\n")

index_file_exists = os.path.isfile("index.txt")

# checking last value of index in order to continue from last position
if index_file_exists:
    with open("index.txt", "r") as index_file:
        last_index = int(index_file.read())
else:
    last_index = 0

print("Output of last_index")
print(last_index)

# iterate over the dataframe
for index, row in df.iloc[last_index:].iterrows():
    print("Ouput of index")
    print(index)
    
    print("Ouput of row")
    print(row)

    #============================================================================================================
    # L O A D    P A G E    O N    S C R A P I N G    B R O W S E R
    #============================================================================================================
    driver.get(row['links'])
    try:
        print("Waiting for Title Element to Load....")
        title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'BookPageTitleSection__title')))
        print("Assigning Title")
        title = title_element.text
    except TimeoutException:
        print(f"Timed out while waiting for title for book {row['links']}, moving on to next book")
        continue
    
    print("Ouput of title")
    print(title)
    #============================================================================================================
    # D E F I N I T I O N    S E C T I O N
    #============================================================================================================
    try:
        print("Waiting for Synopsis and Reviews to load....")
        time.sleep(20)
        synopsis_and_review_list = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Formatted')))
        print(f"The variable length of book {index+1} = ")
        print(len(synopsis_and_review_list))
        synopsis_length = len(synopsis_and_review_list)
        print("Output of synopsis_length")
        print(synopsis_length)
    except TimeoutException:
        print(f"Timed out while waiting for synopsis and review list for book {title}, moving on to next book")
        continue
    
    try:
        page_count_element = driver.find_element_by_css_selector('p[data-testid="pagesFormat"]')
        page_count = page_count_element.text
        
        print("Ouput of page_count")
        print(page_count)
    except NoSuchElementException:
        page_count = ""
    
    try:
        year_element = driver.find_element_by_css_selector('p[data-testid="publicationInfo"]')
        year = year_element.text
        
        print("Ouput of year")
        print(year)
    except NoSuchElementException:
        year = ""
    
    try:
        review_count_element = driver.find_element_by_css_selector('span[data-testid="reviewsCount"]')
        review_count = review_count_element.text
        
        print("Ouput of review_count")
        print(review_count)
        
        # converting string to interger with RegEx
        review_count_match = re.search(r'\d+', review_count)
        if review_count_match:
            review_count = int(review_count_match.group())
            print("Ouput of RegEx: review_count")
            print(review_count)
        else:
            review_count = 0
            print("Ouput of No_RegEx: review_count")
            print(review_count)
            
    except NoSuchElementException:
        review_count = 0
        print("Ouput of No_Count: review_count")
        print(review_count)
    
    try:
        rating_count_element = driver.find_element_by_css_selector('span[data-testid="ratingsCount"]')
        rating_count = rating_count_element.text
        print("Ouput of rating_count")
        print(rating_count)
    except NoSuchElementException:
        rating_count = ""
    
    try:
        rating_element = driver.find_element_by_class_name('RatingStatistics__rating')    
        rating = rating_element.text
        print("Ouput of rating")
        print(rating)
        
    except NoSuchElementException:
        rating = ""
    
    try:
        genre_class_element = driver.find_element_by_class_name('BookPageMetadataSection__genres')
        genres_text_element = genre_class_element.find_elements_by_css_selector('.BookPageMetadataSection__genreButton .Button__labelItem')
        genre = genres_text_element[0].text
        print("Ouput of genre")
        print(genre)
        
    except NoSuchElementException:
        genre = ""
    
    try:
        author_element = driver.find_element_by_class_name('ContributorLink__name')
        author = author_element.text
        print("Ouput of author")
        print(author)

    except NoSuchElementException:
        author = ""
        
    book_reviews = [title, page_count, year, review_count, rating, rating_count, genre, author]
    
    # current row in DataFrame. create these columns
    # and add book information to current dataframe row
    df.at[index, 'page_count'] = page_count
    df.at[index, 'year'] = year
    df.at[index, 'review_count'] = review_count
    df.at[index, 'rating'] = rating
    df.at[index, 'rating_count'] = rating_count
    df.at[index, 'genre'] = genre
    df.at[index, 'author'] = author

    
    # =====================
    # T E S T   P R I N T
    # =====================
    print(f"Book {index}:")
    print(title)
    print("==============")
    print(page_count)
    print("\n")
    print("==============")
    print(year)
    print("\n")
    print("==============")
    print("Total Reviews: ")
    print(review_count)
    print("\n")
    print("==============")
    print(rating)
    print("\n")
    print("==============")
    print(rating_count)
    print("\n")
    print("==============")
    print(genre)
    print("\n")
    print("==============")
    print(author)
    print("\n")
    # ==================================================================
    # R E V I E W S
    # ==================================================================
    # iterating 5 times for reviews offset by 3 
    # list of elements "synopsis_and_review_list" has synopsis
    # and reviews
    review_index = 3
    for i in range(5):
        if review_index < synopsis_length:
            print(f"Review {i + 1}")
            print("==============")
            print(synopsis_and_review_list[review_index].text)

            # saving current review to temporary list
            book_reviews.append(synopsis_and_review_list[review_index].text)
            review_index += 1
        else:
            book_reviews.append("")
    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    with open("reviews.csv", "a", newline='', encoding='utf-8') as reviews_file:
        csv_writer = csv.writer(reviews_file)
        if not reviews_file_exists:
            csv_writer.writerow(['Book Name', 'page_count', 'year', 'review_count', 'rating', 'rating_count', 'genre', 'author', 'Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5'])
            reviews_file_exists = True
        csv_writer.writerow(book_reviews)

    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    # save place of index to file for resuming later
    with open("index.txt", "w") as index_file:
        index_file.write(str(index + 1))

# D O N E    W I T H    S C R A P E R
driver.quit()

if not reviews_file_exists:
    open("reviews.csv", "a", newline='', encoding='utf-8').close()

if not index_file_exists:
    open("index.txt", "a").close()

Does Review CSV File Exists?
False


Output of last_index
0
Ouput of index
0
Ouput of row
links    https://www.goodreads.com/book/show/13771831-b...
Name: 0, dtype: object
Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Godmothers #5
Breaking News
Waiting for Synopsis and Reviews to load....
The variable length of book 1 = 
33
Output of synopsis_length
33
Ouput of page_count
272 pages, Paperback
Ouput of year
First published January 1, 2012
Ouput of review_count
131 reviews
Ouput of RegEx: review_count
131
Ouput of rating_count
1,649 ratings
Ouput of rating
4.17
Ouput of genre
Romance
Ouput of author
Fern Michaels
Book 0:
The Godmothers #5
Breaking News
272 pages, Paperback


First published January 1, 2012


Total Reviews: 
131


4.17


1,649 ratings


Romance


Fern Michaels


Review 1
Not the greatest until about the last third of the book. Maybe my taste in authors is changing.
Review 2
This is the fifth sequel in this series. A little better than the last 

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Black Wall of Silence: A Novel
Waiting for Synopsis and Reviews to load....
The variable length of book 6 = 
6
Output of synopsis_length
6
Ouput of page_count
231 pages, Kindle Edition
Ouput of year
First published March 22, 2015
Ouput of review_count
3 reviews
Ouput of RegEx: review_count
3
Ouput of rating_count
23 ratings
Ouput of rating
4.22
Ouput of author
Paul F Morrissey
Book 5:
The Black Wall of Silence: A Novel
231 pages, Kindle Edition


First published March 22, 2015


Total Reviews: 
3


4.22


23 ratings





Paul F Morrissey


Review 1
This is not a who done it, because you know who done it: Father Zach's so-called friend, Bishop Peter, transferred a priest with history of pedophilia to a poor parish and let the chips fall where they will. And fall, they did. Why? Because that's the way it's always been done. Protect the institution, and silence the victims. Problem: the offender ended up murdered by 

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Oxenburg Princes #1
The Prince Who Loved Me
Waiting for Synopsis and Reviews to load....
The variable length of book 8 = 
33
Output of synopsis_length
33
Ouput of page_count
355 pages, Mass Market Paperback
Ouput of year
First published September 23, 2014
Ouput of review_count
304 reviews
Ouput of RegEx: review_count
304
Ouput of rating_count
1,991 ratings
Ouput of rating
3.80
Ouput of genre
Historical Romance
Ouput of author
Karen Hawkins
Book 7:
The Oxenburg Princes #1
The Prince Who Loved Me
355 pages, Mass Market Paperback


First published September 23, 2014


Total Reviews: 
304


3.80


1,991 ratings


Historical Romance


Karen Hawkins


Review 1
This is a (very loose) retelling of Cinderella in a historical romance format.

Bronwyn is a minor Scottish lady. She, her eccentric inventor father, her stepmother, and her two younger stepsisters are living in genteel poverty - able to keep on a servant and a co

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Secret [With Bonus Material]
Waiting for Synopsis and Reviews to load....
The variable length of book 9 = 
33
Output of synopsis_length
33
Ouput of page_count
327 pages, ebook
Ouput of year
First published June 6, 2015
Ouput of review_count
539 reviews
Ouput of RegEx: review_count
539
Ouput of rating_count
3,787 ratings
Ouput of rating
4.20
Ouput of genre
M M Romance
Ouput of author
Kindle Alexander
Book 8:
Secret [With Bonus Material]
327 pages, ebook


First published June 6, 2015


Total Reviews: 
539


4.20


3,787 ratings


M M Romance


Kindle Alexander


Review 1
Re-Read:

Sometimes I find I just need some Kindle Alexander in my life. Her books are simply guilty pleasure reads for me. I just eat them up.



Enjoyed it almost as much the second time around!

First read:



Oh man, I think this might be my favorite Kindle Alexander yet!!!!

That blurb, no lie….



….had me shaking in my boots. It screamed angst a

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Poetic Edda Volume II Mythological Poems Edited and translated with introduction and commentary by The late Ursula Dronke
Waiting for Synopsis and Reviews to load....
The variable length of book 10 = 
33
Output of synopsis_length
33
Ouput of page_count
458 pages, Hardcover
Ouput of year
First published January 1, 1969
Ouput of review_count
42 reviews
Ouput of RegEx: review_count
42
Ouput of rating_count
754 ratings
Ouput of rating
4.34
Ouput of genre
Poetry
Ouput of author
Ursula Dronke
Book 9:
The Poetic Edda Volume II Mythological Poems Edited and translated with introduction and commentary by The late Ursula Dronke
458 pages, Hardcover


First published January 1, 1969


Total Reviews: 
42


4.34


754 ratings


Poetry


Ursula Dronke


Review 1
It feels undoubtedly strange to review this book. In a manner of speaking, like e.g. rating the Bible; the book had (and still has) such an enormous impact on the Weste

...

Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Hand You're Dealt
Waiting for Synopsis and Reviews to load....
The variable length of book 462 = 
33
Output of synopsis_length
33
Ouput of page_count
176 pages, Hardcover
Ouput of year
First published September 24, 2008
Ouput of review_count
39 reviews
Ouput of RegEx: review_count
39
Ouput of rating_count
156 ratings
Ouput of rating
3.65
Ouput of genre
Young Adult
Ouput of author
Paul Volponi
Book 461:
The Hand You're Dealt
176 pages, Hardcover


First published September 24, 2008


Total Reviews: 
39


3.65


156 ratings


Young Adult


Paul Volponi


Review 1
Huck Porter is a poker prodigy, having been groomed by his father from a young age (much to the chagrin of his mother). Every year the local church hosts an annual Texas Hold'em tournament and sends the winner to Las Vegas to play in a high stakes tournament. The standing agreement is that whatever money is won will be split between the winner and the strug

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Letters and Papers from Prison
Waiting for Synopsis and Reviews to load....
The variable length of book 463 = 
33
Output of synopsis_length
33
Ouput of page_count
437 pages, Paperback
Ouput of year
First published January 1, 1951
Ouput of review_count
201 reviews
Ouput of RegEx: review_count
201
Ouput of rating_count
3,456 ratings
Ouput of rating
4.31
Ouput of genre
Theology
Ouput of author
Dietrich Bonhoeffer
Book 462:
Letters and Papers from Prison
437 pages, Paperback


First published January 1, 1951


Total Reviews: 
201


4.31


3,456 ratings


Theology


Dietrich Bonhoeffer


Review 1
Another fantastic one for my quote series:

“Please don’t ever get anxious or worry about me, but don’t forget to pray for me – I’m sure you don’t. I am so sure of God’s guiding hand that I hope I shall always be kept in that certainty. You must never doubt that I’m traveling with gratitude and cheerfulness along the road where I’

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Betrayed by P. C. Cast l Summary & Study Guide
Waiting for Synopsis and Reviews to load....
The variable length of book 464 = 
3
Output of synopsis_length
3
Ouput of page_count
37 pages, Kindle Edition
Ouput of year
First published April 1, 2012
Ouput of review_count
0 reviews
Ouput of RegEx: review_count
0
Ouput of rating_count
7 ratings
Ouput of rating
3.43
Ouput of author
BookRags
Book 463:
Betrayed by P. C. Cast l Summary & Study Guide
37 pages, Kindle Edition


First published April 1, 2012


Total Reviews: 
0


3.43


7 ratings





BookRags


Ouput of index
464
Ouput of row
links    https://www.goodreads.com/book/show/721012.The...
Name: 464, dtype: object
Waiting for Title Element to Load....
Assigning Title
Ouput of title
The Legend of Sleepy Hollow and Other Stories
Waiting for Synopsis and Reviews to load....
The variable length of book 465 = 
33
Output of synopsis_length
33
Ouput of page_count
368 pages, P

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Fade
Waiting for Synopsis and Reviews to load....
The variable length of book 466 = 
33
Output of synopsis_length
33
Ouput of page_count
320 pages, Paperback
Ouput of year
First published October 1, 1988
Ouput of review_count
267 reviews
Ouput of RegEx: review_count
267
Ouput of rating_count
2,719 ratings
Ouput of rating
3.78
Ouput of genre
Young Adult
Ouput of author
Robert Cormier
Book 465:
Fade
320 pages, Paperback


First published October 1, 1988


Total Reviews: 
267


3.78


2,719 ratings


Young Adult


Robert Cormier


Review 1
As someone who wasn't IN LOVE with Cormier's book The Chocolate War as many others, I was simply blown away by FADE. A perfect example of magical realism - the real world here is painstakingly detailed, rendered so beautifully and realistically that when the magic is sprinkled in it, too, seems completely and utterly real. Plus, this is the type of book with writing that was so beautif

Waiting for Title Element to Load....
Assigning Title
Ouput of title
Birth Control
Waiting for Synopsis and Reviews to load....
The variable length of book 469 = 
4
Output of synopsis_length
4
Ouput of page_count
128 pages, Hardcover
Ouput of year
First published January 1, 2009
Ouput of review_count
1 review
Ouput of RegEx: review_count
1
Ouput of rating_count
1 rating
Ouput of rating
1.00
Ouput of author
Lauri S. Friedman
Book 468:
Birth Control
128 pages, Hardcover


First published January 1, 2009


Total Reviews: 
1


1.00


1 rating





Lauri S. Friedman


Review 1
I was disapointed. I had enjoyed the last opposing viewpoints I read but this one each side cited different statistics for the same thing, and also I noticed all the viewpoints against birth control were written by religious people. They especially made me mad with the one that said pharmacists should have the right to refuse to fill birth control prescriptions because everyone has the freedom to choose. However they 